In [6]:
import pandas as pd
from konlpy.tag import Okt
okt = Okt()
import re
import os
import datetime
import json
import numpy as np

In [7]:
def cleaner2(sent):
    try:
        sent = re.sub('<[^<]+?>', '', sent).strip()
        sent = re.sub('[!"#$%&\'()*,./:;<=>?@[\\]^_`{|}~]+ *',' ',sent)
    except : sent = ''
    return sent
def todatetime(x):
    return datetime.datetime.strptime(str(x),'%Y%m%d')
def todatetime2(x):
    return datetime.datetime.strptime(str(x),'%Y-%m-%d')
def joiner(x1,x2,x3):
    return ' '.join([x1,x2,x3])

### 기사 데이터 준비

#### 기사데이터 로드

In [3]:
dl = pd.Series(os.listdir('/opt/notebooks/AA008/ws_fn/데이터수집/data/빅카인즈'))
dlb = dl.apply(lambda x : x.endswith('csv'))
dl = dl[dlb].reset_index(drop=True)

FileNotFoundError: [Errno 2] No such file or directory: '/opt/notebooks/AA008/ws_fn/데이터수집/data/빅카인즈'

In [5]:
dfs = []
for i in dl:
    tempdf = pd.read_csv('/opt/notebooks/AA008/ws_fn/데이터수집/data/빅카인즈/' + i)
    dfs.append(tempdf)

In [6]:
dfs = pd.concat(dfs,axis=0).reset_index(drop=True)

#### 경제 기사만 가져오기

In [7]:
classes = dfs['구분명'].apply(lambda x : x.split('>')[0])

In [8]:
cldict = dict()
j=0
for i in np.unique(classes):
    cldict[i] = j
    j+=1
cln = [cldict[i] for i in classes]
dfs['cl'] = pd.Series(cln)

In [76]:
cldict

{'IT_과학': 0, '경제': 1, '국제': 2, '문화': 3, '사회': 4, '스포츠': 5, '정치': 6, '지역': 7}

In [285]:
econ = dfs[dfs['cl']==1].reset_index(drop=True) #take only economic articles

In [286]:
bol = econ['구분명'].apply(lambda x:'경제>경제일반' in x)
bol2 = econ['구분명'].apply(lambda x:'문화'not in x)
bol3 = econ['구분명'].apply(lambda x:'날씨'not in x)
bol4 = econ['구분명'].apply(lambda x:'교육_시험'not in x)
bol5 = ((bol+0)+(bol2+0)+(bol3+0)+(bol4+0))==4

In [287]:
econ = econ[bol5].reset_index(drop=True)

#### OKT 토큰화

In [274]:
econ['new기준일자'] = econ['기준일자'].apply(lambda x:todatetime(x))

In [275]:
econ = econ[['new기준일자','기사제목','기사내용']]

In [276]:
econ = econ.sort_values('new기준일자').reset_index(drop=True)

In [277]:
econ['기사제목'] = econ['기사제목'].apply(lambda x:cleaner2(x))
econ['기사내용'] = econ['기사내용'].apply(lambda x:cleaner2(x))

In [ ]:
article = econ[['new기준일자','기사제목']]

In [ ]:
article['okt'] = article['기사제목'].apply(lambda x:okt.morphs(x))

In [ ]:
article.to_csv('article.csv')

### 리포트 준비

#### 리포트 데이터 로드 및 OKT 토큰화

In [4]:
df2 = pd.read_csv('종합의견_20210813.csv').iloc[:,1:]

In [8]:
df2['new날짜'] = df2['날짜'].apply(lambda x:todatetime2(x))

In [9]:
df2 = df2[['new날짜','제목','보고서제목','보고서요약']]

In [10]:
df2 = df2.sort_values('new날짜').reset_index(drop=True)

In [11]:
df2['제목'] = df2['제목'].apply(lambda x:cleaner2(x))
df2['보고서제목'] = df2['보고서제목'].apply(lambda x:cleaner2(x))
df2['보고서요약'] = df2['보고서요약'].apply(lambda x:cleaner2(x))

In [12]:
report = df2.apply(lambda x:joiner(x['제목'],x['보고서제목'],x['보고서요약']),axis=1)

In [13]:
report = pd.concat([df2['new날짜'],report],axis=1)
report.columns = ['date','report']

In [14]:
report['okt'] = report['report'].apply(lambda x:okt.morphs(x))

KeyboardInterrupt: 

In [16]:
report

,date,report
0,2010-12-13,POSCO 005490 현대미포조선 010620 투자의견 Hold 및 목표주가...
1,2010-12-13,LG 003550 LG 003550 목표주가를 기존 115 000원에서 13...
2,2010-12-13,현대모비스 012330 현대모비스 012330 목표주가를 325 000 원으로 ...
3,2010-12-14,동국제강 001230 동국제강 001230 투자의견 시장수익률상회 유지 목표...
4,2010-12-14,하나금융 086790 하나금융 086790 ㆍ 주가 강세에 따른 선순홖 구조...
...,...,...
56692,2021-08-12,넷마블 251270 3 21-1Q22 몇가지 기대 신작 일정은 감안 필요 넷마블 2...
56693,2021-08-13,컴투스 078340 2Q21 어닝쇼크 백년전쟁 부진 크로니클 일정 연 하나금융지주...
56694,2021-08-13,컴투스 078340 뼈아픈 백년전쟁의 부진 코스맥스 192820 중국 온라인 트렌드...
56695,2021-08-13,오리온 271560 원가 부담 지속 비용 절감 이상의 자구책 필요 카페24 0420...


In [ ]:
report.to_csv('report.csv')